In [459]:
import pandas as pd
import os
import sys
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [460]:
df = pd.read_csv("lens-data-2.csv") 
df.drop(["#"], axis=1,inplace=True)
df.columns= df.columns.str.lower()
df1 = df[df['inventors'].notnull()]
maindf = df1[df1['applicants'].notnull()]
maindf.reset_index(inplace=True)
maindf.head()

,index,jurisdiction,kind,display key,lens id,publication date,publication year,application number,application date,earliest priority date,title,applicants,inventors,owners,url,legal status
0,0,US,A1,US 2017/0147742 A1,046-338-410-657-855,2017-05-25,2017,US 201515321739 A,2015-06-19,2014-06-27,SYSTEMS AND METHODS FOR SYNTHETIC BIOLOGY DESI...,UNIV NANYANG TECH,JAYARAMAN PREMKUMAR;;POH CHUEH LOO;;WANG HUI JUAN,NANYANG TECHNOLOGICAL UNIVERSITY (2015-10-29),https://lens.org/046-338-410-657-855,DISCONTINUED
1,1,US,B2,US 10607716 B2,068-534-912-401-872,2020-03-31,2020,US 201615064206 A,2016-03-08,2011-06-06,Synthetic biology tools,UNIV CALIFORNIA,LOU CHUNBO;;MOON TAE SEOK;;RHODIUS VIRGIL;;STA...,THE REGENTS OF THE UNIVERSITY OF CALIFORNIA (2...,https://lens.org/068-534-912-401-872,ACTIVE
2,2,US,A1,US 2009/0061520 A1,075-918-668-399-614,2009-03-05,2009,US 93516407 A,2007-11-05,2006-11-03,Synthetic Biology Vectors,UNIV MICHIGAN,WOOLF PETER;;NINFA ALEX;;MAYO AVRAHAM E;;SELIN...,THE REGENTS OF THE UNIVERSITY OF MICHIGAN (200...,https://lens.org/075-918-668-399-614,INACTIVE
3,3,WO,A2,WO 2021/035115 A2,009-086-590-277-234,2021-02-25,2021,US 2020/0047330 W,2020-08-21,2019-08-21,GROWING PROGRAMMABLE ENZYME-FUNCTIONALIZED AND...,MASSACHUSETTS INST TECHNOLOGY;;IMP COLLEGE LONDON,LU TIMOTHY;;TANG TZU-CHIEH;;OTT WOLFGANG;;ELLI...,NaN,https://lens.org/009-086-590-277-234,PENDING
4,4,US,A1,US 2021/0158197 A1,038-794-758-325-674,2021-05-27,2021,US 202017104817 A,2020-11-25,2019-11-26,BIOLOGY EXPERIMENT DESIGNS,UNIV CALIFORNIA,COSTELLO ZACHARY;;RADIVOJEVIC TIJANA;;MARTIN H...,THE REGENTS OF THE UNIVERSITY OF CALIFORNIA (2...,https://lens.org/038-794-758-325-674,PENDING


In [461]:
def make_dict(df, column):
    finaldict = {}
    for index, url in enumerate(df['url']):
        finaldict[url] = df[column][index].split(';;')
    return finaldict


In [462]:
def make_list(df, column):
    finallist = []
    for i in df[column]:
            finallist.append(i.split(';;'))
    return finallist

In [463]:
#dictionary of lens patent link, then inventory names
maindict = make_dict(maindf,'inventors')

In [464]:
#list of inventors related to a patent
inventor_list_init = make_list(maindf,'inventors')

In [465]:
def uniquelist(list):
    flist = []
    for l in list:
        for i in l:
            if i in flist:
                pass
            else:
                flist.append(i)
    return flist

In [466]:
#list of all unique inventors
inventorlist = uniquelist(make_list(maindf,'inventors'))
inventorlist[0:10]

['JAYARAMAN PREMKUMAR',
 'POH CHUEH LOO',
 'WANG HUI JUAN',
 'LOU CHUNBO',
 'MOON TAE SEOK',
 'RHODIUS VIRGIL',
 'STANTON BRYNNE',
 'TAMSIR ALVIN',
 'TEMME KARSTEN',
 'VOIGT CHRIS']

In [448]:
print(inventorlist[19])

GILBERT CHARLES


In [467]:
newdf = pd.DataFrame({'jurisdiction' : maindf['jurisdiction'],
                      'publication year' : maindf['publication year'],
                      'title' : maindf['title']
                      })
newdf.head()

,jurisdiction,publication year,title
0,US,2017,SYSTEMS AND METHODS FOR SYNTHETIC BIOLOGY DESI...
1,US,2020,Synthetic biology tools
2,US,2009,Synthetic Biology Vectors
3,WO,2021,GROWING PROGRAMMABLE ENZYME-FUNCTIONALIZED AND...
4,US,2021,BIOLOGY EXPERIMENT DESIGNS


In [586]:

def datacol(prof):
    #go to profile input
    profile_key = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/nav/ul/li[3]/a")
    profile_key.click()

    #input name
    input = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[1]/label/input")
    input.send_keys(prof)

    #click search
    searchb = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[1]/button").click()
    try: searchb
    except: 
        driver.get(url)
        return()
    
    time.sleep(5)

    try:
        driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[2]/div[1]")
        pass
    except:
        driver.get(url)
        return()
    
    #new link
    try:
        link = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[2]/div[1]/div/a")
    except:
        driver.get(url)
        return()
    linkn = link.get_attribute('href')
    driver.get(linkn)

    #collecting data---

    #waiting for data to load
    time.sleep(5)
    def waiting():
        if driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[1]/a/strong").text == '-':
            time.sleep(2)
            waiting()
        else:
            pass
    waiting()

    #data collection points
    nworks = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[1]/a/strong").text
    colab_ratio = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[3]/a/strong").text
    lens_h_index = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[4]/a/strong").text
    scholar_cites = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[6]/a/strong").text
    patent_cites = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[7]/a/strong").text
    fields = [driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/div").text]
    lensname = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[1]/div/div[2]/div[1]/h1").text

    try: workcheck = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[3]/div[2]/div/div[1]").text
    except: workcheck = 'N/A'

    if workcheck == 'N/A': placeofwork = 'no current information'
    else:
        if 'Present' in workcheck: placeofwork = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[3]/div[2]/div[1]/div[2]/strong").text
        else: placeofwork = 'no current information'
    
    driver.get(url)
    final['name_lens'].append( lensname)
    final['lens_link' ].append( linkn)
    final['number_of_works'].append( nworks)
    final['collaboration_ratio'].append( colab_ratio)
    final['lens_h_index'].append( lens_h_index)
    final['scholar_article_cites'].append( scholar_cites)
    final[ 'patent_cites'].append( patent_cites)
    final['associated_fields'].append( fields)
    final['current_place_of_work'].append(placeofwork)


In [592]:
url = 'https://www.lens.org/'
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

final = {'name_lens' : [], 
        'lens_link' : [], 
        'number_of_works' :[], 
        'collaboration_ratio' : [], 
        'lens_h_index' : [], 
        'scholar_article_cites' : [], 
        'patent_cites' : [], 
        'associated_fields' : [], 
        'current_place_of_work' : []}

for i in inventorlist[0:999]:
        datacol(i)

In [593]:
final

{'name_lens': ['Premkumar Jayaraman',
  'Poh Theng (Beatrice) Loo',
  'Hui-Juan Wang',
  'Chunbo Lou',
  'Tae Seok Moon',
  'Tamsir Ariyadi',
  'Andrew Temme',
  'Birgit Voigt',
  'avi mayo',
  'Rachel Selinsky',
  'Qi Xiu',
  'Timothy Lu',
  'Tzu-Chieh Tang',
  'Tom Ellis',
  'Gilbert Charles Morgan',
  'Ryan Costello',
  'Tijana Radivojevic',
  'Hector Garcia Martin',
  'VITO PAOLO PASTORE',
  'Simone Bianco',
  'Marshall Wallace',
  'Dazhao Xu',
  'Liu Chuanming',
  'Tingfu Liu',
  'Joyce A Hogue',
  'luca ciotti',
  'J L Baker',
  'J L Baker',
  'Alexander Katzmann',
  'Nane Kratzke',
  'Moritz Mühlberg',
  'Mallory Suhling',
  'Luis Ruben Soenksen',
  'Nicolaas Angenent-Mari',
  'Alexander Alexander',
  'James Collins',
  'george church',
  'Katherine Collins',
  'Diogo M Camacho',
  'Li-Chi Chiang',
  'Chang-Chieh Cheng',
  'Rajiv Bharadwaj',
  'Anup K Singh',
  'chandrasekaran',
  'Lily Hillson',
  'Roy Pardee',
  'Masoud Norouzi',
  'Mauricio Elzo',
  'Dennis S',
  'jean-marc N

In [594]:
newdf = pd.DataFrame(final)
newdf

,name_lens,lens_link,number_of_works,collaboration_ratio,lens_h_index,scholar_article_cites,patent_cites,associated_fields,current_place_of_work
0,Premkumar Jayaraman,https://www.lens.org/lens/orcid/0000-0003-3279...,7,57%,6,413,22,"[Biology , Microbiology , Synthetic biology , ...",no current information
1,Poh Theng (Beatrice) Loo,https://www.lens.org/lens/orcid/0000-0002-6859...,7,43%,6,104,0,"[Business , Complaint , Marketing , Service (b...",Xiamen University Malaysia
2,Hui-Juan Wang,https://www.lens.org/lens/orcid/0000-0002-6110...,7,86%,4,42,0,"[Chemistry , Crystallography , Materials scien...",no current information
3,Chunbo Lou,https://www.lens.org/lens/orcid/0000-0002-7941...,46,84%,19,"2,024",57,"[Biology , Computational biology , Genetics , ...",Shenzhen Institutes of Advanced Technology
4,Tae Seok Moon,https://www.lens.org/lens/orcid/0000-0001-8373...,61,52%,28,"3,704",117,"[Biology , Synthetic biology , Chemistry , Bio...",no current information
...,...,...,...,...,...,...,...,...,...
758,Rumei Chen,https://www.lens.org/lens/orcid/0000-0003-4449...,7,100%,4,70,0,"[Biology , Cell biology , Gene , Botany , Endo...",no current information
759,Suzhen Li,https://www.lens.org/lens/orcid/0000-0001-7498...,2,100%,2,7,0,"[Aleurone , Alternative medicine , Archaeology...",no current information
760,Zhizhen Li,https://www.lens.org/lens/orcid/0000-0003-2538...,2,50%,1,5,0,"[Alloy , Amorphous solid , Chemical engineerin...",no current information
761,Zhanyong Guo,https://www.lens.org/lens/orcid/0000-0002-2143...,52,90%,16,713,3,"[Chemistry , Chitosan , Nuclear chemistry , An...",no current information


In [596]:
newdf.to_excel("output.xlsx")